In [ ]:
# https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f
# ID3

Attribute vs Measure
https://analystanswers.com/what-is-a-data-attribute-definition-types-examples/

https://github.com/DunZhang/DPTrees

In [1]:
import pandas as pd 
import numpy as np 
from collections import Counter, defaultdict
import random
import math

In [ ]:
datasetiris = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
row_names = ["sepal length", "sepal width", "petal length", "petal width", "class"]

irisdata = pd.read_csv(datasetiris, names=row_names,na_values=[' ?'])
irisdata['class'].replace(to_replace=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], value=[1, 2, 3], inplace=True)
irisdata["class"] = irisdata["class"].astype(int)
Diris = irisdata.to_numpy()
# Diris = Diris.astype(str)
Diris

In [5]:
len(Diris)

150

In [6]:
dataset = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
row_names = ["Age", "Workclass", "Fnlwgt", "Education", "EducationNum", "MaritalStatus",
        "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss",
        "HoursPerWeek", "Country", "Income"]
us_adult_income = pd.read_csv(dataset, names=row_names,na_values=[' ?'])
us_adult_income["Income"] = pd.Categorical(us_adult_income["Income"])
us_adult_income["Income"] = us_adult_income["Income"].cat.codes
us_adult_income["Income"] = 1 - us_adult_income["Income"]

us_adult_income.head()

,Age,Workclass,Fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [7]:
random.seed(42)
adult2 = us_adult_income.sample(frac=0.05, random_state=42)
adult2.shape

(1628, 15)

In [8]:
adult25 = adult2.copy()
adult25["Workclass"] = pd.Categorical(adult25["Workclass"])
adult25["Workclass"] = adult25["Workclass"].cat.codes

adult25["Education"] = pd.Categorical(adult25["Education"])
adult25["Education"] = adult25["Education"].cat.codes

adult25["MaritalStatus"] = pd.Categorical(adult25["MaritalStatus"])
adult25["MaritalStatus"] = adult25["MaritalStatus"].cat.codes

adult25["Occupation"] = pd.Categorical(adult25["Occupation"])
adult25["Occupation"] = adult25["Occupation"].cat.codes

adult25["Relationship"] = pd.Categorical(adult25["Relationship"])
adult25["Relationship"] = adult25["Relationship"].cat.codes

adult25["Race"] = pd.Categorical(adult25["Race"])
adult25["Race"] = adult25["Race"].cat.codes

adult25["Gender"] = pd.Categorical(adult25["Gender"])
adult25["Gender"] = adult25["Gender"].cat.codes

adult25["Country"] = pd.Categorical(adult25["Country"])
adult25["Country"] = adult25["Country"].cat.codes

In [9]:
adult23 = adult25[["Age", "Education", "Occupation", "Gender", "Income"]] # Education, Occupation and Gender are discrete. 
adult23

,Age,Education,Occupation,Gender,Income
14160,27,15,0,0,1
27048,45,11,3,0,1
28868,29,9,3,1,0
5667,30,9,6,0,1
7827,29,15,2,1,1
...,...,...,...,...,...
17491,19,11,11,0,1
18954,50,8,9,1,1
15112,44,11,2,1,1
5477,19,15,9,1,1


In [10]:
adult23 = adult23[adult23["Occupation"] != 1]
adult23 = adult23[adult23["Occupation"] != 8]
adult23["Occupation"].value_counts()

 9     232
 3     208
 2     198
 0     190
 11    184
 7     150
-1     101
 6      95
 13     87
 5      61
 4      47
 10     36
 12     34
Name: Occupation, dtype: int64

In [11]:
adult23 = adult23[adult23["Education"] != 13]
adult23 = adult23[adult23["Education"] != 3]
adult23["Education"].value_counts()

11    504
15    362
9     256
12     95
8      67
1      63
7      62
0      46
5      38
14     36
6      32
2      24
10     16
4      15
Name: Education, dtype: int64

In [12]:
np.random.seed(97)
D2 = adult23.to_numpy()
D2 = D2.astype(str)
np.random.shuffle(D2)

In [13]:
D4 = D2.astype(str)
D4

array([['34', '9', '9', '0', '1'],
       ['26', '14', '9', '1', '0'],
       ['57', '4', '2', '1', '1'],
       ...,
       ['38', '9', '11', '1', '0'],
       ['41', '12', '9', '0', '1'],
       ['23', '15', '5', '1', '1']], dtype='<U21')

In [14]:
# make sure both test and training data have all classes in them
# the code will not run if D88 and D99 do not print the same arrays
D22 = D2[400:]

In [15]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,2]).astype(int))

array([-1,  0,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13])

In [16]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,2]).astype(int))

array([-1,  0,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13])

In [17]:
D88 = D22[:len(D22)//2]
np.sort(np.unique(D88[:,1]).astype(int))

array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15])

In [18]:
D99 = D22[len(D22)//2:]
np.sort(np.unique(D99[:,1]).astype(int))

array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15])

https://github.com/loginaway/DecisionTree

In [19]:
import numpy as np

class Node(object):
    '''
    Basic element for decision tree.
    Initialization: to initialize a tree node, at least feature_name is needed.
    Connection: to connect different nodes, there are two situations.
        1, if the parent node is discrete (i.e. node.discrete==True), 
            then 
            >> node[feature_value]=childnode 
            will do the connection.
        2, if the parent node is continuous, then 
            >> node['<=']=childnode 
            or 
            >> node['>']=childnode 
            will assign its children. To be specific, 
            node['<=']=childnode means if the feature value is bigger than 
            node.threshold, then the workflow goes left to the childnode, and vice versa.
    
    Classification: if you have constructed a tree, you may want to classify a 
        certain group of data. Simply call the root node will do this task, i.e.
        >> root(data)
        where data should be a dictionary containing the corresponding key and value.
        And this method will return the classification result.

    Visualization: A naive visualization is implemented here. 
        For example, you may visualize a subtree rooted at 'node' by
        >> print(node)
        The tree structure will then be printed on the console.
    '''
    def __init__(self, feature_name='', discrete=True, threshold=0, depth=0, isLeaf=False, classification=None):
        # for discrete node: the next node can be retrieved by self.map[feature_value]
        # for continuous node: by self.map['<='] (when value <= self.threshold)
        #                       by self.map['>'] (when value > self.threshold)

        self.map=dict()

        self.discrete=discrete
        self.feature_name=feature_name
        self.isLeaf=isLeaf
        if isLeaf:
            self.classification=classification
        if not discrete:
            self.threshold=threshold
        self.depth=depth
        self._class_counts = defaultdict(int)
        self._noisy_class_counts = None

    def __setitem__(self, key, value):
        self.map[key]=value

    def __getitem__(self, key):
        return self.map.get(key)

    def __getdepth__(self):
        return self.depth 

    def increment_class_count(self, class_value):
        self._class_counts[str(class_value)] += 1

    def __noisy_class_counts__(self, epsilon, class_values):
        
        ''' Add noise to the class counts of this leaf. Not performed on parents. '''
        if not self._noisy_class_counts and not self._children: # to make sure this code is only run once per leaf
            
            counts = {}
            for val in class_values:

                if val in self._class_counts:
                    # Laplace noise added to THE CLASS COUNTS
                    counts[val] = max( 0, int(self._class_counts[val] + np.random.laplace(scale=float(1./epsilon))) )
                else: # original count was 0
                    counts[val] = max( 0, int(np.random.laplace(scale=float(1./epsilon))) )

            self._noisy_class_counts = counts

            # what I'm assuming is equation (10)
            self._signal_to_noise = (epsilon * 1.0 * sum([v for k,v in counts.items()])) / (math.sqrt(2*1.0) * len(counts)) # enS / (sqrt(2n) * num_classes)
         
            #print("self._signal_to_noise = {}".format(self._signal_to_noise))
            return 0
        else:
            return 1 # we're summing the redundant calls

    def __call__(self, data): # returns the classification result! 
        
        '''
        This method should be used on the root to predict its classification.
        data: a dict with its key being the features and value being 
        the corresponding value.
        '''

        # print("Inside __call__ method of Node")

        #print("data: ")
        #print(data)

        #print("feature name: ")
        #print(self.feature_name)

        #print("the map right now")
        #print(self.map)

        if self.isLeaf:
            return self.classification

        # if the node has a discrete feature, use __getitem__ to find the next node
        # then call the next node.
        if self.discrete:
            return self.map[data[self.feature_name]](data)

        else:
        # node is not discrete: print the self.feature_name
            #print("continuous name")
            #print(self.feature_name)

            if data[self.feature_name].astype(np.float32)>self.threshold:
                return self.map['>'](data)
            else:
                return self.map['<='](data)

    def __str__(self):
        '''
        This method is designed for printing the subtree rooted at the current node.
        To print the tree, try print(self).
        '''
        hierarchy={}
        stack=[(0, 0, self)]
        count=0
        while stack:
            # BFS
            (layer_index, node_index, current)=stack.pop(0)
            layer_index+=1

            hierarchy[layer_index]=hierarchy.get(layer_index, [])
            hierarchy[layer_index].append((node_index, current.feature_name,\
                    [(str(current.classification),)] if current.isLeaf else \
                     [(str(key), '' if current.discrete else str(current.threshold), item.feature_name) for key, item in current.map.items()]))
            # print(hierarchy)

            if current.isLeaf:
                continue
            for _, item in current.map.items():
                count+=1
                stack.append((layer_index, count, item))
            # print(item_index_map)

        totallist=[]
        for layer, layer_content in hierarchy.items():
            rowlist=[]
            for i in layer_content:
                rowlist.append('['+i[1]+']'+': {'+', '.join([' '.join(item) for item in i[2]])+'}')
            rowstr=' +++ '.join(rowlist)
            totallist.append(rowstr)

        return '============Root===========\n'+'\n\n'.join(totallist)+'\n============Leaf==========='

In [20]:
herlist = D2[:, 4]
unique, counts = np.unique(herlist, return_counts=True)
print(np.asarray((unique, counts)).T)

[['0' '399']
 ['1' '1217']]


In [21]:
dm = 5
sum(2/(dm - i) for i in range(0, (dm - 2))) + (2 / (dm - (dm-1) + 2 )) + 1 # assuming we reach maximum depth

3.2333333333333334

In [43]:
# currently includes methods as:
        # _init_  : _trees
        # get_domains
        # reduce trees
        # evaluate_accuracy_with_voting

class DPRF_Forest:  
    def __init__(self, 
                 training_nonprocessed,
                 training_dataframe, 
                 training_data, # 2D list of the training data where the columns are the attributes, and the first column is the class attribute
                 # test_data, # T
                 epsilon, # epsilon, Budget, B, the total privacy budget
                 f, # n of attributes to be split used by each dctree f
                 max_depth, # max tree depth,
                 feature_discrete 
                 ):

        # Forest = {}
        self._trees = []

        self.epsilon = epsilon

        self.max_depth = max_depth

        self.feature_discrete = feature_discrete


        ''' Some initialization '''
        # Attribute set F
        attribute_values = self.get_domains(training_data)

        # Class values set
        class_values = [str(y) for y in list(set([x[len(x)-1] for x in training_data]))]
        attribute_indexes = [int(k) for k,v in attribute_values.items()]


        #print(attribute_values)
        #print("------------")
        #print(class_values)
        #print("------------")
        #print(attribute_indexes)
        #print("-----------")

        
        # |D|

        valid_attribute_sizes = [[int(k),len(v)] for k,v in attribute_values.items()]

        # Number of trees t
        num_trees = len(attribute_indexes)

        #majority vote!
        if num_trees % 2 == 0:
            num_trees = num_trees + 1
 

        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] )

        # e = B / t
        epsilon_per_tree = epsilon / float(num_trees)

        ''' Calculating minimum support threshold '''
        # estimated_support_min_depth = dataset_size / (average_domain**2) # a large number
        # estimated_support_max_depth = dataset_size / (average_domain ** (len(attribute_indexes)/2)) # max tree depth is k/2 # a small number
        # epsilon_per_tree = epsilon / float(num_trees)
        # min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER # the minimum support in order for S>N


        # while estimated_support_max_depth > min_support: # then we can have more trees
        #    num_trees += 1
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        # while min_support > estimated_support_min_depth and num_trees>1: # then we need to have less trees
        #    num_trees, valid_attribute_sizes, estimated_support_min_depth = self.reduce_trees(num_trees, valid_attribute_sizes, dataset_size)
        #    epsilon_per_tree = epsilon / float(num_trees)
        #    min_support = len(class_values) * math.sqrt(2) * (1/epsilon_per_tree) * SNR_MULTIPLIER


        print("NUM TREES = {} & EPSILON PER TREE = {}".format(num_trees, epsilon_per_tree))

        # ? 
        root_attributes = []

       
        for a in attribute_indexes:
            if a in [x[0] for x in valid_attribute_sizes]:
                root_attributes.append(a) # OR: [index, support, gini]

        print("root attributes")
        print(root_attributes)

        # for treeId = 1,2 ... t do:
        for t in range(num_trees):

            if not root_attributes:
                root_attributes = attribute_indexes[:]

            # randomly extract |D| samples from D w/ a bagging algo?
            root = random.choice(root_attributes)
            root_attributes.remove(root)

            print("what is root attributes: ")
            print(root_attributes)


            # TREE BUILDTREE(D, A, C, eps, f, Dm)
            # D :

            # attribute_indexes, - A_ind
            # attribute_values, A - not exactly the same after all! 
            # root, 
            # class_values, 
            # feature_discrete
            # treetype 
            # dataset_size, 
            # epsilon_per_tree

            A={name: [i] for i, name in enumerate(training_dataframe.columns[:-1])}

            # {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True, 'CapitalLoss': False}
            # if the column in the original dataframe is integervalued: False, 
            # this needs to be passed to the algorithm... 
            #feature_discre={name: False if isinstance(training_nonprocessed[name].iloc[0], np.integer) else True for name in training_dataframe.columns[:-1]}

            # print("feature discrete")
            # print(feature_discre)

            #print("---------------")
            #print("A: ")
            #print(A)
            #print("---------------")
            #print("attribute values: ")
            #print(attribute_values)
            #print("---------------")
            #print("---------------")


            # |D|
            print(len(training_data))
            l = random.sample(range(0, len(training_data)), 1180)
            tr_data = training_data[l]

            dataset_size = len(tr_data)

            tree = Tree_DPDT(attribute_indexes, 
                            A, # original style A
                            attribute_values, # new style attribute vals
                            root, 
                            class_values, 
                            self.feature_discrete, 
                            'IGGR',  # treetype
                            dataset_size, 
                            epsilon_per_tree,
                            max_depth) 

            # each tree needs to be trained on a different subsample of the training data! 
            tree.train(training_data[:len(tr_data)//2], 1)
           
            # TO EVOLVE HERE 

            # attribute_name = ["Age", "Education", "Occupation", "Gender", "CapitalLoss"]

            #num_unclassified = tree.filter_training_data_and_count(training_data, epsilon_per_tree, class_values)
            
            #if num_unclassified > 0:
            #    print("number of unclassified records = {}".format(num_unclassified))
            

            print("currently pruning! ")
            # D2[:len(D2)//2], D2[len(D2)//2:]
            tree.prune(training_data[:len(tr_data)//2], tr_data[len(tr_data)//2:])

            # FOREST = FOREST U TREE
            self._trees.append(tree)

        # print(self._trees)
        # print(len(self._trees))
        # print("succesfully ended building tree dictionary")


    def get_domains(self, data):
        attr_domains = {}
        transData = np.transpose(data)
        for i in range(0,len(data[0]) - 1):
            attr_domains[str(i)] = [str(x) for x in set(transData[i])]
            print("original domain length of categorical attribute {}: {}".format(i, len(attr_domains[str(i)])))
        return attr_domains


    def reduce_trees(self, num_trees, valid_attribute_sizes, dataset_size):
        num_trees -= 1
        largest_attribute = sorted(valid_attribute_sizes, key=lambda x:x[1], reverse=True)[0]
        #print("Removing att{} with domain size {}".format(largest_attribute[0], largest_attribute[1])) 
        new_valids = [ x for x in valid_attribute_sizes if x[0] != largest_attribute[0] ]
        average_domain = np.mean( [x[1] for x in valid_attribute_sizes] ) # average with all attributes
        narrowed_average_domain = np.mean([x[1] for x in valid_attribute_sizes if x[0] in [y[0] for y in new_valids] ]) # average without the big attributes
        estimated_support_squared = dataset_size / (average_domain * narrowed_average_domain)
        return num_trees, new_valids, estimated_support_squared


    # GET MAJORITY LABELS (Forest, T, C) equivalent from pseudocode
    def evaluate_accuracy_with_voting(self, records, class_index=4):

        ''' Calculate the Prediction Accuracy of the Forest. '''
        actual_labels = [x[class_index] for x in records]

        predicted_labels = []
        leafs_not_used = 0
        count_of_averages_used = 0

        class_counts = defaultdict(list)

        # FOR EACH RECORD X IN DATASET DO
        # Strange for we need to access individual predictions here...
        for rec in records:

            h = []

            class_value_fractions = defaultdict(list)

            # FOR EACH TREE DO
            for tree in self._trees:

                # GET PREDICTED CLASSIFICATION RESULT for a single record
                # node, leaf_not_used = tree._classify(tree._root_node, rec)

                result = tree.pred(np.array([rec]))

                h.append(*result)

                #unique, counts = np.unique(h, return_counts=True)
                #class_counts = dict(zip(unique, h))

                #print("class counts")
                #print(class_counts)

                #leafs_not_used += leaf_not_used

                #support = float(sum([v for k,v in noisy_class_counts.items()]))

                #for k,v in noisy_class_counts.items():
                #    class_value_fractions[k].append(v/support)
                    
            #best_confidences = {}

            #for k,lis in class_value_fractions.items():
            #   best_confidences[k] = max(lis)
            #best = [None, 0.0]

            #for k,class_best in best_confidences.items():
            #    if class_best > best[1]:
            #        best = [k, class_best]
            
            # average_used = False

            # majority vote
            #for k, class_best in best_confidences.items():

            #    if class_best == best[1] and k != best[0]:

                    # average_used = True
                   
                    #orig_average = np.mean(class_value_fractions[best[0]])

                    # contender_average = np.mean(class_value_fractions[k])
            
                    # if contender_average > orig_average:
            #            best = [k, class_best]

            # count_of_averages_used += 1 if average_used else 0

            # majority vote

            # IndexError: index 1 is out of bounds for axis 0 with size 1
            hh = [int(*el) for el in h]

            print("votes: ")
            print(hh)

            c = Counter(hh)
            ans = c.most_common(1)

            predicted_labels.append(ans[0][0])

        print("data and trees looped trhough!")

        # counts how many records we got right!

        
        print(type(predicted_labels[0]))
        print(type(actual_labels[0]))

        # print(zip(predicted_labels, actual_labels))

        counts = Counter([x == y for x, y in zip(map(str, predicted_labels), actual_labels)])
        
        # acc = float(counts[True]) / len(records)
        
        return float(counts[True]) / len(records)
        

In [45]:
import sys
import numpy as np
import math
#from Node import Node

# set the maximal recursion limits here.
sys.setrecursionlimit(10000)

# to be modified to single class

# attribute_indexes, - A_ind
# attribute_values, A
# root, 
# class_values, 
# feature_discrete
# treetype 
# dataset_size, 
# epsilon_per_tree


class Tree_DPDT(DPRF_Forest):

    '''
    The main class of decision tree.
    '''
    def __init__(self, A_ind, A, attribute_values, root, class_values, feature_discrete, treetype, dataset_size, epsilon_per_tree, max_depth):
        '''
        attribute_values : attribute_values
        A : orignal
        feature_discrete: a dict with its each key-value pair being (feature_name: True/False),
            where True means the feature is discrete and False means the feature is 
            continuous. 
        type: ID3/C4.5/CART
        pruning: pre/post
        '''

        self.A=A
        self.attribute_values=attribute_values
        self.A_ind=A_ind
        self.feature_discrete= feature_discrete
        self.treeType=treetype
        self.leaf_count=0
        self.tmp_classification=''
        self.class_values=class_values
        self._root_node = root
        self.tree=None
        self.dataset_size=dataset_size
        self.epsilon=epsilon_per_tree
        self.dm = max_depth
        self.current_IGGR = 1

        self.eu = epsilon_per_tree / sum(2/(self.dm - i) for i in range(0, (self.dm - 2))) + (2 / (self.dm - (self.dm-1) + 2 )) + 1
        self.ei = self.eu
        self.ei1 = 0 
        self.ei2 = self.ei

    def Entropy(self, list_of_class):
        '''
        Compute the entropy for the given list of class.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        'duck': 2/3, 'dolphin': 1/3, so the entropy for this array is 0.918
        '''
        count={}
        for key in list_of_class:
            count[key]=count.get(key, 0)+1

            
        frequency=np.array(tuple(count.values()))/len(list_of_class)
        return -1*np.vdot(frequency, np.log2(frequency))

    def Information_Gain(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Gain.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Gain for this example is 0.2516.
        '''
        sec2=np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
        return self.Entropy(list_of_class)-sec2


    def Information_Ratio(self, list_of_class, grouped_list_of_class):
        '''
        Compute the Information Ratio.
        list_of_class: an array of classification labels, e.g. ['duck', 'duck', 'dolphin']
        grouped_list_of_class: the list of class grouped by the values of 
            a certain attribute, e.g. [('duck'), ('duck', 'dolphin')].
        The Information_Ratio for this example is 0.2740.
        '''
        tmp=np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
        # Here we assume instrinsic_value is SplitInformation! 
        intrinsic_value=-1*np.vdot(tmp, np.log2(tmp))

        return self.Information_Gain(list_of_class, grouped_list_of_class) / intrinsic_value

    def IGGR(self, list_of_class, grouped_list_of_class, a1, a2, eps, curIGGR):

        #print("eps at IGGR: ", eps)

        def infoGainAndGainRatio(list_of_class, grouped_list_of_class):
            sec2 = np.sum([len(item)*self.Entropy(item) for item in grouped_list_of_class])/len(list_of_class)
            infoGain =  self.Entropy(list_of_class)-sec2
            tmp = np.array([len(item)/len(list_of_class) for item in grouped_list_of_class])
        
            intrinsic_value = -1*np.vdot(tmp, np.log2(tmp))

            GainRatio = self.Information_Gain(list_of_class, grouped_list_of_class)/ intrinsic_value

            return infoGain, GainRatio, intrinsic_value 

        infoGainRes, GainRatioRes, intrinsic_value_res = infoGainAndGainRatio(list_of_class, grouped_list_of_class)

        sol1 = a1 * infoGainRes + a2 * GainRatioRes
        sol2 = (a1 + ( a2 / intrinsic_value_res) ) * infoGainRes 

        # print("relevant data for sensitivity")
        # print(list_of_class) classes 
        # print(grouped_list_of_class)
        # print(curIGGR, " and ", sol2)

        sens_delta = 1 # ?

        exp_noise = np.random.exponential( (eps * abs(sol2)) / 2 * sens_delta * abs(curIGGR) )

        #exp_n = 6

        # return sol2, exp_noise
        return exp_noise


    def orderByIGGR(self, D, A, a1, a2, eps):

        '''
        Return the order by Information Gain or Information Ratio.
        by: 'Gain', 'Ratio'.
        For the definition of D and A, see the remark in method 'fit'.
        '''

        # print("epsilon at IGGR order: ", eps)
        tmp_value_dict=dict()

        # target_function1 = self.Information_Gain
        # target_function2 = self.Information_Ratio
        # target_function3 = self.IGGR
        
        for attr, info in A.items():
            possibleVal=np.unique(D[:, info[0]])
            # if the continuous attribute have only one possible value, then 
            # choosing it won't improve the model, so we abandon it.
            if len(possibleVal)==1:
                continue

            if self.feature_discrete[attr] is True:
                # discrete
                if len(info)<2:
                    A[attr].append(possibleVal)
                # retrieve the grouped list of class
                grouped_list_of_class=[]
                for val in possibleVal:
                    indexes=np.argwhere(D[:, info[0]]==val)
                    grouped_list_of_class.append(D[indexes, -1].flatten())
                IC_value = self.IGGR( D[:, -1], grouped_list_of_class, a1, a2, eps, self.current_IGGR )
                self.current_IGGR = IC_value
                tmp_value_dict[attr]=IC_value

            else:

                # continuous

                split_points=(possibleVal[: -1].astype(np.float32)+possibleVal[1:].astype(np.float32))/2
                maxMetric=-1
                for point in split_points:

                    # modified now to float comparisons
                    # causes the code to crash to max recursion
                    # smaller_set = D[np.argwhere(D[:, info[0]].astype(float) <= point ), -1].flatten()
                    # info[0] = 4, point = 951.0
                    # bigger_set = D[np.argwhere(D[:, info[0]].astype(float) > point ), -1].flatten()
                    
                    smaller_set = D[np.argwhere(D[:, info[0]] <= str(point) ), -1].flatten()
                    # info[0] = 4, point = 951.0
                    bigger_set = D[np.argwhere(D[:, info[0]] > str(point) ), -1].flatten()

                    # compute the metric
                    # list_of_class, grouped_list_of_class, a1, a2
                    IC_tmp = self.IGGR( D[:, -1], (smaller_set, bigger_set), a1, a2, eps, self.current_IGGR )
                    self.current_IGGR = IC_tmp

                    if IC_tmp>maxMetric:
                        maxMetric=IC_tmp
                        threshold=point

                    #print("attribute was: ", attr)
                    #print("sets (small and big)")
                    #print(smaller_set)
                    #print(bigger_set)

                    # IC_tmp=0.0

                # set the threshold
                if len(info)<2:
                    A[attr].append(threshold)
                else:
                    A[attr][1]=threshold
                tmp_value_dict[attr]=maxMetric

        # find the attribute with the max tmp_value_dict value
        attr_list=list(tmp_value_dict.keys())
        attr_list.sort(key=lambda x: tmp_value_dict[x])

        return attr_list

    def chooseAttribute(self, D, A, eps):

        '''
        Choose an attribute from A according to the metrics above.
        For the definition of D and A, see method 'fit'.
        Different principal for different tree types:
        ID3: choose the attribute that maximizes the Information Gain.
        C4.5: 
            1, choose those attributes whose Information Gain are above average.
            2, choose the one that maximizes the Gain Ratio from these attributes.
        CART: choose the attribute that minimizes the Gini Index.
        IG_GR: 
        '''

        # print("epsilon passed: ", eps)

        if self.treeType=='ID3':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')
            return attr_list[-1]

        if self.treeType=='C4.5':
            attr_list=self.orderByGainOrRatio(D, A, by='Gain')

            # for C4.5, we choose the attributes whose Gain are above average
            # and then order them by Ratio.

            sub_A={key: A[key] for key in attr_list}
            attr_list=self.orderByGainOrRatio(D, sub_A, by='Ratio')

            return attr_list[-1]

        if self.treeType=='IGGR':

            attr_list=self.orderByIGGR(D, A, 0.4, 0.6, eps)

            return attr_list[-1]

    # tree.train(training_data[:len(training_data)//2], 1)
    def train(self, D, depth):
        self.tree=self.fit(D, self.A, depth)

    def fit(self, D, A, depth):

        '''
        Train the tree.
        To save the training result:
        >> self.tree=self.fit(D, A)
        D: the training set, a size [m, n+1] numpy array (with str type elements), 
            where m is the number of training data and n is the number of attributes.
            The last column of D is the classifications (or labels).
        A: the attributes set. It is a dict with its structure being like 
            {attr_name: [index_in_D_columns, possibleVal_or_threshold], ...}
            attr_name: name of the attribute
            index_in_D_columns: the corresponding index of the attribute in ndarray D (starting from 0)
            possibleVal_or_threshold: 
                ###################################################
                ## This value may not always be available in A   ##
                ## it is added after 'chooseAttribute' is called ##
                ## And it will be updated after each call        ##
                ###################################################
                1, if the attribute is discrete, then it is a ndarray containing all possible values 
                    of this attribute.
                2, if the attribute is continuous, then possibleVal_or_threshold is the most recent 
                    threshold.
        '''

        # termination conditions 

        # the training set is empty
        if len(D)==0:
            node=Node(feature_name='leaf-'+str(self.leaf_count), depth=depth, isLeaf=True, \
                classification=self.tmp_classification)
            self.leaf_count+=1
            return node

        # only one type of classification is left 
        if len(np.unique(D[:, -1]))<=1:
            node=Node(feature_name='leaf-'+str(self.leaf_count), depth=depth, isLeaf=True, \
                classification=D[0, -1])
            self.leaf_count+=1
            return node

        if len(A)==0 or len(np.unique(D[:, :-1], axis=0))<=1:

            count_dict={}

            for key in D[:, -1]:

                count_dict[key]=count_dict.get(key, 0)+1

            most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]

            node=Node(feature_name='leaf-'+str(self.leaf_count), depth=depth, isLeaf=True, \
                classification=most_frequent)

            self.leaf_count+=1
            return node 

        count_dict={}

        # the count query usese the Laplace mechanism to add noise to the class count
        for key in D[:, -1]:
            count_dict[key] = count_dict.get(key, 0)+1 + np.random.laplace(scale=1/self.eu)

        print("count dictionary: ")
        print(count_dict)
        # {'1': 449, '0': 162}
        # {'0': 4, '1': 5}
        
        most_frequent=sorted(D[:, -1], key=lambda x: count_dict[x])[-1]
        print("most frequent : ")
        print(most_frequent)
        # count dictionary: 
        #{'0': 4, '1': 1}
        # most frequent : 
        # 0

        self.tmp_classification=most_frequent

        # choose target attribute
        target_attr=self.chooseAttribute(D, A, self.eu)
        # print(target_attr)


        # generate nodes for each possible values of the target attribute if it's discrete
        # related information is stored in A[target_attr][1] now, 
        # since we have called chooseAttribute at least once.
        info=A[target_attr]

        if self.feature_discrete[target_attr]:

            node=Node(feature_name=target_attr, discrete=True, depth=depth, isLeaf=False)
            # generate nodes for each possible value
            
            for possibleVal in info[1]:

                keys=set(A.keys()).difference({target_attr})
                # connect node to its child
                tmp_D=D[np.argwhere(D[:, info[0]]==possibleVal), :]
                tmp_A={key: A[key] for key in keys}

                node[possibleVal] = self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), tmp_A, depth+1)
        
        else:
            # generate two nodes for the two classification if it's continuous
            # continuous
            threshold=info[1]
            node=Node(feature_name=target_attr, discrete=False, threshold=threshold, depth=depth, isLeaf=False)

            tmp_D=D[np.argwhere(D[:, info[0]]<=str(threshold)), :]
            node['<=']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A, depth+1)

            tmp_D=D[np.argwhere(D[:, info[0]]>str(threshold)), :]
            node['>']=self.fit(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), A, depth+1)
            
        
        return node

    def prune(self, training_D, testing_D):
        self.post_prune(training_D, testing_D, self.A, current=self.tree)

    def post_prune(self, training_D, testing_D, A, current=None, parent=None):
    
        '''
        self.tree is required.
        This method conducts the post-pruning to enhance the model performance.
        To make sure this method will work, set 
        >> current=self.tree
        when you call it.
        '''

        self.current_accuracy=self.evaluate(testing_D, A)

        count_dict={}
        if len(training_D)==0:
            return 

        for key in training_D[:, -1]:
            count_dict[key] = count_dict.get(key, 0)+1
        most_frequent=sorted(training_D[:, -1], key=lambda x: count_dict[x])[-1]

        leaf_parent=True
        for key, node in current.map.items():
            if not node.isLeaf:
                leaf_parent=False

                # Recursion, DFS
                if node.discrete:
                    tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]==key), :]
                else:
                    if key=='<=':
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]<=str(node.threshold)), :]
                    else:
                        tmp_D=training_D[np.argwhere(training_D[:, A[current.feature_name][0]]>str(node.threshold)), :]
                self.post_prune(tmp_D.reshape((tmp_D.shape[0], tmp_D.shape[2])), testing_D, A, parent=current, current=node)
        
        tmp_node=Node(feature_name='leaf-'+str(self.leaf_count), isLeaf=True, classification=most_frequent)
        
        if parent:
            # when current node is not the root
            for key, node in parent.map.items():
                if node==current:
                    parent.map[key]=tmp_node
                    saved_key=key
                    break

            # compare the evaluation, if it is enhanced then prune the tree.
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                parent.map[saved_key]=current
                
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return
        else:

            # when current node is the root
            saved_tree=self.tree
            self.tree=tmp_node
            tmp_accuracy=self.evaluate(testing_D, A)
            if tmp_accuracy<self.current_accuracy:
                self.tree=saved_tree
            else:
                self.current_accuracy=tmp_accuracy
                self.leaf_count+=1
            return

    def pred(self, D):

        return self.predict(D, self.A)
    
    def eval(self, D):
        return self.evaluate(D, self.A)

    def predict(self, D, A):
        '''
        Predict the classification for the data in D.
        For the definition of A, see method 'fit'. 
        There is one critical difference between D and that defined in 'fit':
            the last column may or may not be the labels. 
            This method works as long as the feature index in A matches the corresponding
            column in D.
        '''

        # why the loop can not be dismissed?
        row, _= D.shape # for the entire testing data 
        pred = np.empty((row, 1), dtype=str)
        tmp_data={key: None for key in A.keys()}

        # print("the tree is a: ", type(self.tree))
        # the tree is a:  <class '__main__.Node'>

        for i in range(len(D)): # only 1 row?
            for key, info in A.items():

                tmp_data[key] = D[i, info[0]]

            #print("data given to tree: ")
            #print(tmp_data)
            # {'Age': '20', 'Education': '8', 'Occupation': '4', 'Gender': '1'}

            # but self.tree is evidently initialized as none?

            pred[i] = self.tree(tmp_data)
        
      
        return pred
    
    def evaluate(self, testing_D, A):
        '''
        Evaluate the performance of decision tree. (Accuracy)
        For definition of testing_D and A, see 'predict'.
        However, here the testing_D is required to be labelled, that is, its last column 
        should be labels of the data.
        '''
        true_label=testing_D[:, -1]
        pred_label=self.predict(testing_D, A)
        
        success_count=0
        for i in range(len(true_label)):
            if true_label[i]==pred_label[i]:
                success_count+=1

        return success_count/len(true_label)

In [ ]:
Diris.astype('<U21')

In [25]:
len(D2[400:])

1216

In [39]:
l = random.sample(range(0, 1216), 1170)
D2[400:][l]

array([['58', '15', '4', '1', '0'],
       ['29', '11', '4', '1', '1'],
       ['48', '15', '-1', '0', '1'],
       ...,
       ['40', '6', '6', '1', '1'],
       ['25', '15', '9', '0', '1'],
       ['17', '1', '11', '0', '1']], dtype='<U21')

In [46]:

# Input:
    # Training set D 
    # Privacy budget B (epsilon)

    # Test Set T (?)  (Maybe?)

    # Number of attributes to be split used by each dec tree f 
    # Max tree depth d_m

B = 0.2 # epsilon
f = 3
dm = 5

feature_discrete_adult = {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True}
diff_priv_forest = DPRF_Forest(adult2, adult23, D2[400:], B, f, dm, feature_discrete_adult)


#feature_discrete_iris =  {'sepal length': False, 'sepal width': False, 'petal length': False, 'petal width': False}
#diff_priv_forest = DPRF_Forest(irisdata, irisdata, Diris.astype('<U21'), B, f, dm, feature_discrete_iris)

#num_trees = len(our_diff_priv_forest._trees)
#av_prunings = np.mean([x._prunings for x in our_diff_priv_forest._trees])
#av_tree_size = np.mean([x._id-x._prunings+1 for x in our_diff_priv_forest._trees])
#accuracy, leafs_not_most_confident, votes_requiring_average = diff_priv_forest.evaluate_accuracy_with_voting(adult23, class_index=0) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

# feature discrete
# {'Age': False, 'Education': True, 'Occupation': True, 'Gender': True}

original domain length of categorical attribute 0: 64
original domain length of categorical attribute 1: 14
original domain length of categorical attribute 2: 13
original domain length of categorical attribute 3: 2
NUM TREES = 5 & EPSILON PER TREE = 0.04
root attributes
[0, 1, 2, 3]
what is root attributes: 
[0, 1, 3]
1216
count dictionary: 
{'1': 486.33019000136113, '0': 145.61692330874362}
most frequent : 
1
count dictionary: 
{'1': 437.1211119227594, '0': 138.86157097295916}
most frequent : 
1
count dictionary: 
{'1': 420.29502644214875, '0': 133.7784183889884}
most frequent : 
1
count dictionary: 
{'1': 412.87007461329347, '0': 148.3735174432803}
most frequent : 
1
count dictionary: 
{'1': 158.1001881439807, '0': 24.61085283942968}
most frequent : 
1
count dictionary: 
{'1': 62.87013348021825, '0': 12.592455870154325}
most frequent : 
1
count dictionary: 
{'1': 56.42584344251147, '0': 10.766699757900753}
most frequent : 
1
count dictionary: 
{'1': 39.825031817443765, '0': 12.546137

In [41]:
#g = D2[400:][:,-4]
#unique, counts = np.unique(g, return_counts=True)
#print(np.asarray((unique, counts)).T)

In [47]:
accuracy = diff_priv_forest.evaluate_accuracy_with_voting(D2[400:][:,0:5], class_index=4) # confidence count includes all trees (before voting)
print("accuracy = {}".format(accuracy))

votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[0, 0, 0, 0, 0]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0, 0, 0]
votes: 
[0, 0, 0, 0, 0]
votes: 
[1, 1, 1, 1, 1]
votes: 
[1, 1, 1, 1, 1]
votes: 
[0, 0, 0

In [ ]:
# something's not right with the voting system! 
#accuracy = diff_priv_forest.evaluate_accuracy_with_voting(Diris.astype('<U21'), class_index=4) # confidence count includes all trees (before voting)
#print("accuracy = {}".format(accuracy))

In [461]:
# %history -g -f hybridTree2019.ipynb